In [1]:
import sys
from io import BytesIO
from pathlib import Path
from multiprocessing import Pool
import pandas as pd

In [2]:
sys.path.append('../src')
from utils import run_cmd

In [3]:
dirpath = Path('/media/NGS/SRA_1/NCBI_Listeria_monocytogenes_SRA/Filter')
outpath = Path('/media/NGS/SRA_1/NCBI_Listeria_monocytogenes_SRA/LisSero')

In [4]:
cmds = []
for i in dirpath.iterdir():
    outfile = outpath/(i.stem + '.txt')
    cmd = f"conda run -n lissero lissero {i} > {outfile}"
    cmds.append(cmd)

In [7]:
with Pool(64) as p:
    try:
        p.map(run_cmd, cmds)
        p.close()
        p.join()
    except KeyboardInterrupt:
        p.terminate()

In [9]:
dirpath = Path('/media/NGS/SRA_1/NCBI_Listeria_monocytogenes_SRA/LisSero')

In [34]:
data = []
for file in dirpath.iterdir():
    with open(file) as handle:
        next(handle)
        value = next(handle).strip().split('\t')
        data.append(value)

In [37]:
df = pd.DataFrame(data, columns=['ID', 'SEROTYPE', 'PRS', 'LMO0737', 'LMO1118', 'ORF2110', 'ORF2819', 'COMMENT'])

In [41]:
df['ID'] = df['ID'].apply(lambda x: Path(x).stem)

In [43]:
df.to_csv('/media/NGS/SRA_1/NCBI_Listeria_monocytogenes_SRA/LisSero.txt', sep='\t', index=False)